# Heading 1

In [ ]:
# this is where you should start coding

### imports


In [ ]:
import os
import tarfile
import requests
import torch
from torch.utils.data import Dataset
import pandas as pd
import cv2
import os
import torch.nn as nn
import torch.nn.functional as F
import math
import tqdm
from sklearn.model_selection import train_test_split
import copy
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch.optim as optim

### model definition


In [ ]:
#

class Net(nn.Module):
  def __init__(self, n_classes):
    super().__init__()

    self.conv = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=4, stride=1, padding=1)

    self.fc1 = nn.Linear(8 * 6 * 7, n_classes)
    self.fc2 = nn.Linear(n_classes, 1)

  def forward(self, x):
    x = x.unsqueeze(1)
    x = self.conv(x)
    x = F.relu(x)
    x = x.view(x.size(0), -1)

    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    return x


model = nn.Sequential(
    nn.Linear(8, 24),
    nn.ReLU(),
    nn.Linear(24, 12),
    nn.ReLU(),
    nn.Linear(12, 6),
    nn.ReLU(),
    nn.Linear(6, 1)
)

In [ ]:
n_epochs = 100
batch_size = 10

### train model using train and validation datasets

In [ ]:
# loss function and optimizer
loss_fn = nn.MSELoss()  # mean square error
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# train-test split of the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, shuffle=True)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

train_size = int(0.8 * len(dataset))
test_size = int(0.1 * len(dataset))
val_size = int(0.1 * len(dataset))
train_set, test_set, val_set = torch.utils.data.random_split(dataset, [train_size, test_size, val_size])

y_train = train_set.pop('price')
y_test = test_set.pop('price')
y_val = val_set.pop('price')

x_train = torch.tensor(train_set, dtype=torch.float32)
x_test = torch.tensor(test_set, dtype=torch.float32)
x_val = torch.tensor(val_set, dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)
y_val = torch.tensor(y_val, dtype=torch.float32).reshape(-1, 1)

# Hold the best model
best_mse = np.inf   # init to infinity
best_model = None
mse_store = []
loss_store = []
eval_epochs = 0
batch_start = torch.arange(0, len(X_train), batch_size)

# training loop
for epoch in range(n_epochs):

    model.train()

    with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
        bar.set_description(f"Epoch {epoch}")

        for start in bar:

            # take a batch
            X_batch = X_train[start:start+batch_size]
            y_batch = y_train[start:start+batch_size]

            # forward pass
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)

            # backward pass
            optimizer.zero_grad()
            loss.backward()

            # update weights
            optimizer.step()

            # print progress
            bar.set_postfix(mse=float(loss))

    # evaluate accuracy at end of each epoch
    model.eval()
    y_pred = model(x_val)

    eval_epochs += 1

    mse = loss_fn(y_pred, y_val)

    mse = float(mse)
    mse_store.append(mse)
    loss_store.append(loss.item())

    print(f"Epoch: {epoch}, Train Loss: {loss.item()} Eval Loss: {mse}")

    if mse < best_mse:
        best_mse = mse
        best_model = copy.deepcopy(model.state_dict())

# Save your model's weights
model.load_state_dict(best_model)
torch.save(model.state_dict(), "model.pth")

### plot training and eval loss values




In [ ]:
# restore model and return best accuracy
model.load_state_dict(best_model)

In [ ]:
# Plot the training and validation loss
plt.plot(eval_epochs, loss_store, label='Training loss')
plt.plot(eval_epochs, mse_store, label='Validation loss')
plt.legend()
plt.show()

# testing model on test set


In [ ]:
# Load from your saved model using torch.load
model_state_dict = torch.load("./model.pth")

model = nn.Sequential(
    nn.Linear(8, 24),
    nn.ReLU(),
    nn.Linear(24, 12),
    nn.ReLU(),
    nn.Linear(12, 6),
    nn.ReLU(),
    nn.Linear(6, 1)
)

model.load_state_dict(model_state_dict)

# set model to inference mode
model.eval()
batch_size = 1
preds = []

test_loader = DataLoader(x_test, batch_size=batch_size, shuffle=False)


with torch.no_grad():

  for x in tqdm(test_loader):

    y_pred = model(x)

    preds.append(y_pred)


# Get the true labels for the validation dataset
true_y = torch.tensor(y_test)
preds = torch.tensor(preds)

accuracy = (true_y == preds).float().mean().item()
print(f"Accuracy: {accuracy}")